![encodelogo](images/encodelogo.gif)

This notebook demonstrates ingesting an ENCODE bigWig into a TileDB dense array with an Amazon S3 backend.

# Why use TileDB?
With TileDB you gain the ability to quickly query array-structured data using rectangular slices, update existing arrays with new or changed data, and easily optimize your physical data organization for maximizing compression and read performance.

# What is bigWig?
BigWig is a widely used format for representing various types of genetic signals. Applications range from visualization to machine learning. For the file type definition see: https://genome.ucsc.edu/goldenPath/help/bigWig.html.

# How to configure S3 backend
In this example we will use *s3://tile-db-test* as the bucket and *us-west-2* as the region. See https://docs.tiledb.io/en/stable/tutorials/working-with-s3.html for details. 

In [1]:
s3_base_url = 's3://tile-db-test/'

# Open bigWig with pyBigWig
ENCODE bigWig files can be accessed from *s3://encode-public*. *pyBigWig* can open files directly from URLs.

In [2]:
import pyBigWig
bw = pyBigWig.open(
'https://encode-public.s3.amazonaws.com/2019/02/15/d5c222de-f74d-4ac7-b056-b3b5d159c773/ENCFF111MVN.bigWig'
)
bw_base_name = 'ENCFF111MVN'
chrom_info = bw.chroms()

# Write data to tileDB
After setting up AWS_ACCESS_KEY_ID and AWS_SECRET_ACCESS_KEY environment variables, the required configuration is minimal. The only thing that needs to be set is the bucket region. Everything else works out of the box.

In [ ]:
import tiledb
import numpy as np

# Configure the region.
config = tiledb.Config()
config['vfs.s3.region'] = 'us-west-2'

# Set the address for the base url where the arrays will
# be written. There is going to be an array per chromosome.
tiledb_s3_baseaddress = s3_base_url + bw_base_name + '/'

# Start building the TileDB.
ctx = tiledb.Ctx(config)

def write_array_to_tiledb(array, url, ctx, default_tile_size=9000,
                          compressor='gzip', compression_level=-1):
    size = array.shape[0]
    tile_size = min(size, default_tile_size)
    tiledb_dim = tiledb.Dim(
        ctx,
        name='genome_coordinate',
        domain=(0, size - 1),
        tile=tile_size,
        dtype='uint32'
    )
    tiledb_dom = tiledb.Domain(ctx, tiledb_dim)
    tiledb_attr = tiledb.Attr(
        ctx,
        name='signal_value',
        compressor=(compressor, compression_level),
        dtype='float32'
    )
    tiledb_schema = tiledb.ArraySchema(
        ctx,
        domain=tiledb_dom,
        attrs=(tiledb_attr,),
        cell_order='row-major',
        tile_order='row-major'
    )
    tiledb.DenseArray.create(url, tiledb_schema)
    with tiledb.DenseArray(ctx, url, mode='w') as s3array:
        array = array.astype(np.float32)
        s3array[:] = array

To iterate over chromosomes and create the TileDB array in the S3 bucket takes a couple hours. Parallel processing is recommended to ingest a lot of bigWigs.

In [ ]:
for chrom_name, chrom_size in chrom_info.items():
    signal_data = np.zeros(chrom_size, dtype=np.float32)
    signal_data[:] = bw.values(chrom_name, 0, chrom_size)
    chrom_s3_address = tiledb_s3_baseaddress + chrom_name
    write_array_to_tiledb(signal_data, chrom_s3_address, ctx)

We can also just push a single chromosome:

In [ ]:
chr1_name = 'chr1'
chr1_size = chrom_info['chr1']
signal_data = np.zeros(chr1_size, dtype=np.float32)
signal_data[:] = bw.values(chr1_name, 0 chr1_size)
chr1_s3_address = tiledb_s3_baseaddress + chr1_name
write_array_to_tiledb(signal_data, chr1_s3_address, ctx)

# Query data
After pushing the data into TileDB we can randomly access the signal very quickly:

In [7]:
with tiledb.DenseArray(ctx, 's3://tile-db-test/ENCFF111MVN/chr1', mode='r') as s3array:
    data = s3array[1000000:1100000]
data['signal_value'][100:200]

array([7.36847, 7.36847, 7.36847, 7.36847, 7.36847, 7.36847, 7.36847,
       8.42119, 8.42119, 8.42119, 8.42119, 7.36847, 7.36847, 7.36847,
       7.36847, 7.36847, 7.36847, 7.36847, 7.36847, 7.36847, 7.36847,
       7.36847, 7.36847, 7.36847, 7.36847, 6.31576, 6.31576, 6.31576,
       6.31576, 6.31576, 6.31576, 6.31576, 6.31576, 6.31576, 6.31576,
       6.31576, 6.31576, 6.31576, 6.31576, 6.31576, 6.31576, 6.31576,
       6.31576, 6.31576, 6.31576, 6.31576, 6.31576, 6.31576, 6.31576,
       6.31576, 6.31576, 6.31576, 6.31576, 6.31576, 5.26331, 5.26331,
       5.26331, 5.26331, 5.26331, 5.26331, 5.26331, 5.26331, 5.26331,
       5.26331, 5.26331, 5.26331, 5.26331, 5.26331, 5.26331, 5.26331,
       5.26331, 5.26331, 3.74616, 3.74616, 3.74616, 3.74616, 3.74616,
       3.74616, 3.74616, 3.74616, 3.74616, 3.74616, 3.74616, 3.74616,
       3.74616, 3.74616, 3.74616, 3.74616, 3.74616, 3.74616, 3.74616,
       3.74616, 2.99689, 2.99689, 2.24762, 2.24762, 1.49835, 1.49835,
       1.49835, 1.49